**CHEST X-RAY IMAGE CLASSIFICATION ADVANCED DATA SCIENCE CAPSTONE PROJECT**

Paolo Cavadini, February 2021.

Dataset
https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia

Find on GitHub: https://github.com/pcavad/capstone_x_rays

In [ ]:
import os

import keras
from keras.preprocessing import image
from keras import backend as K
from keras.models import Sequential, load_model
from keras import layers
from keras.layers import Input, Dense, Dropout, Flatten, MaxPool2D 
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, SeparableConv2D 
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint,EarlyStopping 
import itertools
import matplotlib.pyplot as plt
plt.style.use('seaborn')
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score 
import seaborn as sns
import tensorflow as tf

In [ ]:
X_train = np.load('./data/X_train.npy')
y_train = np.load('./data/y_train.npy')
X_test = np.load('./data/X_test.npy')
y_test = np.load('./data/y_test.npy')

In [ ]:
input_shape = (150,150,3)

In [ ]:
def modeling(activation_f=None, optimizer_f=None, input_shape_img=None):
    '''
    This function creates the model based on different parameters, complile and save it.
    '''
    model = Sequential()
    
    #1st block
    model.add(Conv2D(32 , (3,3) , strides = 1 , padding = 'same' , activation = activation_f , input_shape = input_shape_img))
    model.add(BatchNormalization())
    model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
    #2nd block
    model.add(Conv2D(64 , (3,3) , strides = 1 , padding = 'same' , activation = activation_f))
    model.add(Dropout(0.1))
    model.add(BatchNormalization())
    model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
    #3rd block
    model.add(Conv2D(64 , (3,3) , strides = 1 , padding = 'same' , activation = activation_f))
    model.add(BatchNormalization())
    model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
    #4th block
    model.add(Conv2D(128 , (3,3) , strides = 1 , padding = 'same' , activation = activation_f))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())
    model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
    #5th block
    model.add(Conv2D(256 , (3,3) , strides = 1 , padding = 'same' , activation = activation_f))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())
    model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))

    model.add(Flatten())
    model.add(Dense(units = 128 , activation = activation_f))
    model.add(Dropout(0.2))
    model.add(Dense(units = 1 , activation = 'sigmoid')) 

    model.compile(optimizer=optimizer_f, loss='binary_crossentropy', metrics=['accuracy']) # compiling ##'MeanSquaredError'
    
    model.save('./data/model.h5') # saving
    
    return model

**TRAINING**

**Fine tuning optimizer and activation function**

In [ ]:
# Looping through different optimizers and activation functions to find the best score.
input_shape_img = (150,150,3)
activation_functions = ['relu', 'sigmoid', 'tanh']
optimizers = ['adam', 'rmsprop']
# best_score = 0
best_accuracy = 0
best_optimizer = None
best_activation_function = None

for activation_function in activation_functions:
    for optimizer_function in optimizers:
        modeling(activation_function, optimizer_function, input_shape) # model function
        model = load_model('./data/model.h5') # load model
        history = model.fit( 
                            X_train, y_train,
                            batch_size=32, #1 epoch (default)
                            validation_split=0.2,
                            verbose=1) # validation set
        if history.history['accuracy'][0] > best_accuracy:
            best_accuracy = history.history['accuracy'][0]
            best_optimizer = optimizer_function
            best_activation_function = activation_function
            model.save('./data/best_model.h5')

print('The best optimizer is {}, the best activation function is {}, for an accuracy of {:.2f}%'
     .format(optimizer_function, best_activation_function, best_accuracy*100))

**We have estimated the best activation function and the best optimizer, which we'll use in the next fine tuning steps.**

**FINE TUNING FEATURES**

**Reduce bias**

In [ ]:
# Finding the proportion between scans with and without pneumonia, and calculating the weights to counter-balance the bias during training.

count_normal = len(y_train[y_train==0]) 
count_pneumonia = len(y_train[y_train==1])
initial_bias = np.log([count_pneumonia / count_normal]) 
print("Initial bias: {:.4f}".format(initial_bias[0]))

train_img_count = len(y_train)
weight_for_0 = (1 / count_normal) * train_img_count / 2.0
weight_for_1 = (1 / count_pneumonia) * train_img_count / 2.0

class_weights = {0: weight_for_0, 1: weight_for_1}

print("Weight for class 0: {:.2f}".format(weight_for_0))
print("Weight for class 1: {:.2f}".format(weight_for_1))

In [ ]:
# Training the model adding class weights

model = load_model('./data/best_model.h5')
history = model.fit(
                    X_train,y_train,
                    batch_size=32,
                    validation_split=0.2,
                    class_weight=class_weights,
                    verbose=1) # 1 epoch (default)
if history.history['accuracy'][0] > best_accuracy:
    best_accuracy = history.history['accuracy'][0]
    print('Accuracy improved: ', best_accuracy*100)
else:
    print('Adding class weights didn\'t improve the accuracy')

**It looks like adding the weights improved our model.**

**TRAINING THE FINALLY OPTIMIZED MODEL (10 epochs)**

**Note about performance:** becasue training over different epochs may become too intensive for my hardware I run the model under IBM Watson Studio where I can setup a Notebook with a proper choice of runtime (more CPUs, more memory). To implement the Notebook I just need to upload the X and Y arrays as file in the Object Cloud store and use them instead of parsing the images with Keras.

**Data augmentation**

Data augmentation means increasing the samples by means of adding scans from different perspectives, which bring the model closer to reality.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        rotation_range = 30,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.2, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip = True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

datagen.fit(X_train)

**Callbacks**

Before training the model is useful to define one or more callbacks:

1. ModelCheckpoint: save a copy of the best performing model when an epoch that improves the metrics ends.

2. EarlyStopping: stop training when the difference between training and validation error starts to increase, instead of decreasing (overfitting).

Optimizing the learning rate:

Learning rate is a descent step which the optimizing algorithms take in order to converge to a local optimum. The learning rate should not be too high to take very large steps nor it should be too small which would not alter the weights and biases. The ReduceLRonPlateau monitors the learning rate and if no improvement is seen for a (patience) number of epochs then the learning rate is reduced by a factor specified as one of the parameters.

In [ ]:
# Initializing callbacks
path = f'./data/model.h5'

# Saves the model in-between epochs when there is an improvement in val_loss
checkpoint = ModelCheckpoint(path,
                                monitor='val_loss',
                                mode="min",
                                save_best_only = True,
                                verbose=1)

# Stops training the model when no improvement in val_loss is observed after set patience
earlystop = EarlyStopping(monitor = 'val_loss', 
                              min_delta = 0, 
                              patience = 4,
                              verbose = 1,
                              restore_best_weights = True)

# Monitors val_accuracy for a set 'patience', then the learning rate is reduced by a factor specified in the parameters
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', ## val_accuracy
                              patience = 2,
                              verbose=1,
                              factor=0.3, # reduction
                              min_lr=0.000001)

# callbacks pipeline
callbacks_pipeline = [checkpoint, earlystop, reduce_lr]

In [ ]:
# Modelling and training using the chosen optimizer, activation function, callbacks and no data augmentation.
filt = np.random.rand(len(X_train))

model = load_model('./data/best_model.h5')
history = model.fit(
#                 X_train,y_train,
                datagen.flow(X_train[filt < 0.8],y_train[filt < 0.8], batch_size = 32),
                batch_size=32,
#                 validation_split=0.2,
                class_weight=class_weights,
                validation_data=datagen.flow(X_train[filt > 0.8], y_train[filt > 0.8]), 
                epochs=10, #10 epochs
                callbacks=callbacks_pipeline,
                verbose=1)

In [ ]:
# Predicting and saving
preds = model.predict(X_test)
np.save('./data/preds.npy',preds)

In [ ]:
# Saving history
df_history=pd.DataFrame(history.history)
df_history.to_csv('./data/training_history.csv', index=False)